In [ ]:
# aoi, mes, qms, pandas 모두 import
import aoi
import mes
import qms
import pandas as pd

In [ ]:
# 예제1) 자동검사기 데이터 불러와서 재단 모델 별로 불량율 계산, 비교하기

# 대상 : SF40 ASG5

# 모델 정보 1: 
  # 슬리팅 1(43인치) : 폭(1125), 장변(956.90), 단변(541.95), 흡수축(0)
  # 슬리팅 2(55인치) : 폭(750), 장변(1229.20), 단변(695.5), 흡수축(0)
# 모델 정보 2:
  # 슬리팅 1(43인치) : 폭(990), 장변(956.90), 단변(541.95), 흡수축(90)
  # 슬리팅 1(43인치) : 폭(990), 장변(956.90), 단변(541.95), 흡수축(90)

# 기간 : 3~5월 (3/1 ~ 5/31)

# 불량 정보 : 전 불량, 마킹 별 수율 + 전체 수율

# 출력 : xlsx 파일

In [ ]:
"""1개 예제 학습"""
lot = '20170301DC01001'
slitting_width = 1125
inch_x1 = 956.90
inch_y1 = 541.95
axis1 = 0
inch_x2 = 1229.20
inch_y2 = 695.50
axis2 = 0 

width, length = aoi.read_lot_info(lot)
data = aoi.read_data(lot, True, True)
slit1, slit2 = aoi.preprocessing.slitting(data, slitting_width)

slit1_ratio = aoi.output.ratio(slit1, slitting_width, length, inch_x1, inch_y1, axis1)
slit2_ratio = aoi.output.ratio(slit2, slitting_width, length, inch_x2, inch_y2, axis2)

print(slit1_ratio, slit2_ratio)

In [ ]:
"""200개 예제 학습"""
#데이터 셋 준비
lot_list = mes.read_lot(20170301, 20170531, 'SF40ASG')
lot_list = lot_list[lot_list['코팅'].str.contains(r'DC')]

model1 = [1125, 956.90, 541.95, 0, 1229.20, 695.5, 0]
model2 = [990, 956.90, 541.95, 90, 956.90, 541.95, 0]
models = [model1, model2]

result = pd.DataFrame([])

#반복문
for lot in lot_list['코팅']:
    for model in models:          
        """1개 예제"""
        lot = lot
        slitting_width = model[0]
        inch_x1 = model[1]
        inch_y1 = model[2]
        axis1 = model[3]
        inch_x2 = model[4]
        inch_y2 = model[5]
        axis2 = model[6]
        
        width, length = aoi.read_lot_info(lot)
        data = aoi.read_data(lot, True, True)
        slit1, slit2 = aoi.preprocessing.slitting(data, slitting_width)

        slit1_ratio = aoi.output.ratio(slit1, slitting_width, length, inch_x1, inch_y2, axis1)
        slit2_ratio = aoi.output.ratio(slit2, slitting_width, length, inch_x2, inch_y2, axis2)
        
        ratio = pd.DataFrame([lot, slit1_ratio, slit2_ratio], index = ['코팅','슬리팅1','슬리팅2']).T
        result = pd.concat([result, ratio])      

In [ ]:
result['total'] = result['슬리팅1'] + result['슬리팅2']
model1 = result.iloc[0::2]
model2 = result.iloc[1::2]

In [ ]:
model2['total'] - model1['total']